<a href="https://colab.research.google.com/github/DajanaSanchez/BirdMigrationClimateAnalysis/blob/main/climate_data_remerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd

# Convert 'climate_merged_final_.csv' to Parquet

climate_final_file = "/content/drive/MyDrive/climate_merged_final.csv"
MODIS_final_file = "/content/drive/MyDrive/MODIS_data.csv"
bird_final_file = "/content/drive/MyDrive/bird_observations.parquet"

# Convert 'climate_merged_final_.csv' to Parquet

df_climate = pd.read_csv(climate_final_file)

# Ensure 4 decimal places for latitude and longitude by converting to string format
df_climate["Latitude"] = df_climate["Latitude"].apply(lambda x: f"{x:.4f}")
df_climate["Longitude"] = df_climate["Longitude"].apply(lambda x: f"{x:.4f}")

# Save as Parquet
df_climate.to_parquet("climate_merged_final.parquet", engine="pyarrow", index=False)

print("climate_merged_final_.csv has been converted to Parquet.")

# Convert 'MODIS_data.csv' to Parquet
df_MODIS = pd.read_csv(MODIS_final_file)

# Ensure 4 decimal places for latitude and longitude by converting to string format
df_MODIS["Latitude"] = df_MODIS["Latitude"].apply(lambda x: f"{x:.4f}")
df_MODIS["Longitude"] = df_MODIS["Longitude"].apply(lambda x: f"{x:.4f}")

# Save as Parquet
df_MODIS.to_parquet("last_version_of_merged_data1.parquet", engine="pyarrow", index=False)

print("MODIS_data.csv has been converted to Parquet.")

<ipython-input-2-205805b9a3fa>:11: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_climate = pd.read_csv(climate_final_file)


climate_merged_final_.csv has been converted to Parquet.
MODIS_data.csv has been converted to Parquet.


In [3]:
df_bird = pd.read_parquet(bird_final_file)

In [7]:
df_bird.describe()

,LATITUDE,LONGITUDE
count,7.498836e+06,7.498836e+06
mean,4.238773e+01,-9.358986e+01
std,6.207601e+00,2.055219e+01
min,1.934370e+01,-1.790633e+02
25%,3.912949e+01,-1.140327e+02
50%,4.240090e+01,-8.596323e+01
75%,4.621175e+01,-7.676376e+01
max,7.315889e+01,1.792826e+02


In [8]:
df_bird.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7498836 entries, 0 to 7498835
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   LATITUDE          float64
 1   LONGITUDE         float64
 2   OBSERVATION DATE  object 
dtypes: float64(2), object(1)
memory usage: 171.6+ MB


In [29]:
#count of observations by OBSERVATION DATE and latitude and longitude
df_bird_grouped = df_bird.groupby(['OBSERVATION DATE', 'LATITUDE', 'LONGITUDE']).size().reset_index(name='Record_Count')

#sort by number of records descending
df_sorted = df_bird_grouped.sort_values(by='Record_Count', ascending=False)

print(df_sorted)

        OBSERVATION DATE   LATITUDE   LONGITUDE  Record_Count
2842156          2024-05  41.627710  -83.191884          4507
2482673          2023-05  41.627710  -83.191884          4357
2152165          2022-05  41.627710  -83.191884          3296
1265699          2019-05  41.627710  -83.191884          3023
1053278          2018-05  41.627710  -83.191884          2472
...                  ...        ...         ...           ...
1215241          2019-03  39.390605  -81.235864             1
1215244          2019-03  39.391458  -81.229118             1
1215245          2019-03  39.392072  -82.139604             1
1215247          2019-03  39.394137  -88.058449             1
3044755          2024-12  64.795810 -147.739360             1

[3044756 rows x 4 columns]


In [32]:
# Standardize column names (if needed)
for df in [df_bird, df_climate, df_MODIS]:
    df.rename(columns={'LATITUDE': 'Latitude', 'LONGITUDE': 'Longitude', 'OBSERVATION DATE': 'Date'}, inplace=True)

# Round coordinates to 4 decimals.
df_bird['Latitude'] = df_bird['Latitude'].round(4)
df_bird['Longitude'] = df_bird['Longitude'].round(4)

df_climate['Latitude'] = df_climate['Latitude'].round(4)
df_climate['Longitude'] = df_climate['Longitude'].round(4)

df_MODIS['Latitude'] = df_MODIS['Latitude'].round(4)
df_MODIS['Longitude'] = df_MODIS['Longitude'].round(4)

# Check Block 2:
print("Bird Columns:")
print(df_bird.columns.tolist())
display(df_bird[['Date','Latitude', 'Longitude']].head(5))

print("Climate Columns:")
print(df_climate.columns.tolist())
display(df_climate[['Date','Latitude', 'Longitude', 'Temp_celsius', 'Precip_mm']].head(5))

print("Land Columns:")
print(df_MODIS.columns.tolist())
display(df_MODIS[['Date','Latitude', 'Longitude', 'Land_Cover_Description']].head(5))

Bird Columns:
['Latitude', 'Longitude', 'Date']


,Date,Latitude,Longitude
0,1995-11,32.0453,-115.9085
1,1998-03,30.4094,-115.9458
2,1998-03,30.4094,-115.9458
3,1998-03,30.4094,-115.9458
4,1998-03,30.4094,-115.9458


Climate Columns:
['Station_ID', 'Date', 'Latitude', 'Longitude', 'Elevation', 'State', 'StationName', 'Temp_celsius', 'Precip_mm', 'TempFlags', 'Precip_MeasurementFlag', 'Precip_QCFlag', 'Precip_SourceFlag', 'Precip_SourceIndex', 'Precip_FirstYear', 'Precip_LastYear']


,Date,Latitude,Longitude,Temp_celsius,Precip_mm
0,1961-01,48.9333,-123.7500,5.01,372.3
1,1962-01,48.9333,-123.7500,3.53,107.4
2,1963-01,48.9333,-123.7500,1.55,33.5
3,1964-01,48.9333,-123.7500,4.56,302.5
4,1965-01,48.9333,-123.7500,2.73,169.4


Land Columns:
['Year', 'Date', 'Latitude', 'Longitude', 'Region', 'Land_Cover_Type', 'Land_Cover_Description']


,Date,Latitude,Longitude,Land_Cover_Description
0,2001-01,83.0000,-170.0000,Water
1,2001-01,83.0000,-169.9500,Water
2,2001-01,83.0000,-169.8999,Water
3,2001-01,83.0000,-169.8499,Water
4,2001-01,83.0000,-169.7998,Water


In [33]:
from scipy.spatial import cKDTree

# Build a KDTree for the climate dataset based on Longitude and Latitude.
climate_tree = cKDTree(df_climate[['Longitude', 'Latitude']].values)

def get_nearest_climate(row):
    """
    For a given bird record, find the nearest climate record.
    Returns a Pandas Series with the temperature and precipitation values.
    """
    # Query the nearest climate record.
    distance, idx = climate_tree.query([row['Longitude'], row['Latitude']], k=1)
    nearest = df_climate.iloc[idx]
    return pd.Series({
        'Temp_celsius': nearest['Temp_celsius'],
        'Precip_mm': nearest['Precip_mm']
    })

# Apply the nearest-neighbor lookup to each bird record.
df_bird_nn = df_bird.copy()
climate_info = df_bird_nn.apply(get_nearest_climate, axis=1)
df_bird_nn = pd.concat([df_bird_nn, climate_info], axis=1)

# Check Block 3:
print("Bird data with nearest climate values:")
display(df_bird_nn.head(5))

Bird data with nearest climate values:


,Latitude,Longitude,Date,Temp_celsius,Precip_mm
0,32.0453,-115.9085,1995-11,18.92,5.8
1,30.4094,-115.9458,1998-03,14.92,0.0
2,30.4094,-115.9458,1998-03,14.92,0.0
3,30.4094,-115.9458,1998-03,14.92,0.0
4,30.4094,-115.9458,1998-03,14.92,0.0


In [35]:
# Build a KDTree for the land dataset based on Longitude and Latitude.
land_tree = cKDTree(df_MODIS[['Longitude', 'Latitude']].values)

def get_nearest_land(row):
    """
    For a given bird record, find the nearest land record.
    Returns the Land_Cover_Description from the nearest record.
    """
    distance, idx = land_tree.query([row['Longitude'], row['Latitude']], k=1)
    nearest = df_MODIS.iloc[idx]
    return nearest['Land_Cover_Description']

# Apply the function to each bird record.
df_bird_nn['Land_Cover_Description'] = df_bird_nn.apply(get_nearest_land, axis=1)

# Check Block 4:
print("Bird data with nearest land cover:")
display(df_bird_nn[['Date', 'Latitude', 'Longitude', 'Land_Cover_Description']].head(5))

Bird data with nearest land cover:


,Date,Latitude,Longitude,Land_Cover_Description
0,1995-11,32.0453,-115.9085,Open Shrublands
1,1998-03,30.4094,-115.9458,Grasslands
2,1998-03,30.4094,-115.9458,Grasslands
3,1998-03,30.4094,-115.9458,Grasslands
4,1998-03,30.4094,-115.9458,Grasslands


In [36]:
# In this approach, df_bird_nn already represents your bird dataset
# enriched with the nearest climate (Temp_celsius, Precip_mm) and land (Land_Cover_Description) values.
df_merged_final = df_bird_nn.copy()

# Check Block 5:
print("Final merged dataset shape:", df_merged_final.shape)
print("Final merged dataset columns:")
print(df_merged_final.columns.tolist())
display(df_merged_final.head(10))

Final merged dataset shape: (7498836, 6)
Final merged dataset columns:
['Latitude', 'Longitude', 'Date', 'Temp_celsius', 'Precip_mm', 'Land_Cover_Description']


,Latitude,Longitude,Date,Temp_celsius,Precip_mm,Land_Cover_Description
0,32.0453,-115.9085,1995-11,18.92,5.8,Open Shrublands
1,30.4094,-115.9458,1998-03,14.92,0.0,Grasslands
2,30.4094,-115.9458,1998-03,14.92,0.0,Grasslands
3,30.4094,-115.9458,1998-03,14.92,0.0,Grasslands
4,30.4094,-115.9458,1998-03,14.92,0.0,Grasslands
5,30.4094,-115.9458,1998-03,14.92,0.0,Grasslands
6,32.0453,-115.9085,2001-03,18.92,5.8,Open Shrublands
7,32.0453,-115.9085,2002-01,18.92,5.8,Open Shrublands
8,31.7670,-116.5970,2010-12,15.87,0.0,Water
9,32.0453,-115.9085,2012-02,18.92,5.8,Open Shrublands


In [37]:
df_merged_final.describe()

,Latitude,Longitude,Temp_celsius,Precip_mm
count,7.498836e+06,7.498836e+06,7.498836e+06,7.498836e+06
mean,4.238773e+01,-9.358986e+01,9.982356e+00,8.516875e+01
std,6.207601e+00,2.055219e+01,9.542358e+00,7.740470e+01
min,1.934370e+01,-1.790633e+02,-9.999000e+01,0.000000e+00
25%,3.912950e+01,-1.140327e+02,3.430000e+00,3.760000e+01
50%,4.240090e+01,-8.596320e+01,1.053000e+01,6.680000e+01
75%,4.621170e+01,-7.676380e+01,1.713000e+01,1.129000e+02
max,7.315890e+01,1.792826e+02,3.538000e+01,1.020600e+03


In [38]:
df_merged_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7498836 entries, 0 to 7498835
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Latitude                float64
 1   Longitude               float64
 2   Date                    object 
 3   Temp_celsius            float64
 4   Precip_mm               float64
 5   Land_Cover_Description  object 
dtypes: float64(4), object(2)
memory usage: 343.3+ MB


In [45]:
#view dataframe ordered by date and latitude and longitude
df_merged_final = df_merged_final.sort_values(by=['Latitude', 'Longitude', 'Date'])
df_merged_final.head(100)

,Latitude,Longitude,Date,Temp_celsius,Precip_mm,Land_Cover_Description
558,19.3437,-96.4642,2017-10,27.87,195.2,Cropland/Natural Vegetation Mosaic
559,19.3437,-96.4642,2017-10,27.87,195.2,Cropland/Natural Vegetation Mosaic
560,19.3437,-96.4642,2017-10,27.87,195.2,Cropland/Natural Vegetation Mosaic
561,19.3437,-96.4642,2017-10,27.87,195.2,Cropland/Natural Vegetation Mosaic
562,19.3437,-96.4642,2017-10,27.87,195.2,Cropland/Natural Vegetation Mosaic
...,...,...,...,...,...,...
2368887,24.5393,-81.8840,2002-07,29.76,34.6,Water
170,24.5394,-112.0609,2024-01,20.36,0.2,Water
2371196,24.5454,-81.7831,2017-04,29.33,197.2,Water
2374496,24.5454,-81.7831,2023-03,29.33,197.2,Water


In [46]:
output_file = "/content/drive/MyDrive/final_merged_dataset.csv"
df_merged_final.to_csv(output_file, index=False)
print(f"Harmonized dataset saved to: {output_file}")

Harmonized dataset saved to: /content/drive/MyDrive/final_merged_dataset.csv
